# The Jansen-Rit Model

In this notebook, the basic use of the implementation of the Jansen-Rit (`jr`) model is presented. 

In the `jr` model, the activity of a particular brain region is defined...

In [ ]:
# change to the root directory of the project
import os
if os.getcwd().split("/")[-1] == "examples":
    os.chdir('..')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob

from neurolib.models.jr import JRModel

import neurolib.utils.loadData as ld
import neurolib.utils.functions as func

# a nice color map
plt.rcParams['image.cmap'] = 'plasma'

## Bifurcation diagram

In [ ]:
model = JRModel()
model.params['duration'] = 4.0*1000

Let's draw a simple one-dimensional bifurcation diagram of this model to orient ourselves in the parameter space

In [ ]:
max_y0 = []
min_y0 = []
max_y1 = []
min_y1 = []
max_y2 = []
min_y2 = []

model.params['sigma_ou'] = 0.0

# these are the different input values that we want to scan
p_inputs = np.linspace(-0.6, 1, 100)
for ext_input in p_inputs:
    model.params['ext_input'] = ext_input
    model.run()
    max_y0.append(np.max(model.y0[0, -int(1000/model.params['dt']):]))
    min_y0.append(np.min(model.y0[0, -int(1000/model.params['dt']):]))
    max_y1.append(np.max(model.y1[0, -int(1000/model.params['dt']):]))
    min_y1.append(np.min(model.y1[0, -int(1000/model.params['dt']):]))
    max_y2.append(np.max(model.y2[0, -int(1000/model.params['dt']):]))
    min_y2.append(np.min(model.y2[0, -int(1000/model.params['dt']):]))

In [ ]:
plt.plot(p_inputs, max_y0, c='k', lw = 2)
plt.plot(p_inputs, min_y0, c='b', lw = 2)
plt.title("Bifurcation diagram of y0 output")
plt.xlabel("Input to p ")
plt.ylabel("Min / max y0 [mV]")

In [ ]:
plt.plot(p_inputs, max_y1, c='r', lw = 2)
plt.plot(p_inputs, min_y1, c='g', lw = 2)
plt.title("Bifurcation diagram of y1 output")
plt.xlabel("Input to p")
plt.ylabel("Min / max y0 [mV]")

In [ ]:
plt.plot(p_inputs, max_y2, c='k', lw = 2)
plt.plot(p_inputs, min_y2, c='b', lw = 2)
plt.title("Bifurcation diagram of y2 output")
plt.xlabel("Input to p")
plt.ylabel("Min / max y2 [mV]")

## Single node simulation

In [ ]:
model = JRModel()
model.params['duration'] = 10000
model.params['sigma_ou'] = 0.14
model.run()

In [ ]:
plt.plot(model.t, model.y0.T, c='k', lw = 2)
plt.xlabel("t [ms]")
plt.ylabel("Activity [kHz]")

In [ ]:
plt.plot(model.t, model.y1.T - model.y2.T, c='k', lw = 2)
plt.title("Local Field Potential")
plt.xlabel("t [ms]")
plt.ylabel("LFP [mV]")

## Brain network

In [ ]:
from neurolib.utils.loadData import Dataset

ds = Dataset("hcp")

In [ ]:
model = JRModel(Cmat = ds.Cmat, Dmat = ds.Dmat)

In [ ]:
model.params['ext_input'] = 0.65
model.params['signalV'] = 0
model.params['duration'] = 20 * 1000
model.params['sigma_ou'] = 0.14

model.run()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(func.fc(model.y0[:, -10000:]))
axs[1].plot(model.t, model.y0[::5, :].T, alpha=0.8)
axs[1].set_xlim(1000, 3000)
model.y0.shape

In [ ]:
scores = [func.matrix_correlation(func.fc(model.y0[:, -int(5000/model.params['dt']):]), fcemp) for fcemp in ds.FCs]
print("Correlation per subject:", [f"{s:.2}" for s in scores])
print("Mean FC/FC correlation: {:.2f}".format(np.mean(scores)))